In [16]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import contractions
from spellchecker import SpellChecker
import gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/djkuralt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/djkuralt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/djkuralt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
sightings_raw_data = pd.read_pickle("sightings.pkl")
sightings_raw_df = pd.DataFrame(sightings_raw_data)
sightings_raw_df.head(5)

,Date_Time,City,State,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary
0,4/23/21 06:30,Blackshear,GA,Circle,9 minutes,Very strange ((NUFORC Note: Rocket launch f...,4/23/21,http://www.nuforc.org/webreports/162/S162815.html,\nVery strangeI have recorded a video of this ...
1,4/23/21 06:00,Mechanicsville,VA,Circle,Seconds,Ball in the sky ((NUFORC Note: Rocket launc...,4/23/21,http://www.nuforc.org/webreports/162/S162814.html,\nBall in the skyObject appears as a white bal...
2,4/23/21 06:00,Vero Beach,FL,Light,5 minutes,I was driving and saw something strange in the...,4/23/21,http://www.nuforc.org/webreports/162/S162822.html,\nI was driving and saw something strange in t...
3,4/23/21 05:59,St. Augustine,FL,Light,3 minutes,2 extremely bright lights appeared over east c...,4/23/21,http://www.nuforc.org/webreports/162/S162824.html,\n2 extremely bright lights appeared over east...
4,4/23/21 05:58,Durham,NC,Cone,>5 minutes,A cone of light coming from the sky unlike any...,4/23/21,http://www.nuforc.org/webreports/162/S162819.html,\nA cone of light coming from the sky unlike a...


In [3]:
sightings_raw_df.isna().any()

Date_Time         False
City              False
State             False
Shape             False
Duration          False
Summary           False
Posted            False
Detail_Link       False
Detail_Summary     True
dtype: bool

In [4]:
# the rows with no Detail_Summary entry
sightings_raw_df[sightings_raw_df.Detail_Summary.isna()]

,Date_Time,City,State,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary
35635,1/3/15 20:05,Melbourne Beach,FL,Fireball,5 minutes,"Burning object seen falling, then glowing red,...",1/7/15,http://www.nuforc.org/webreports/116/S116518.html,None
35652,1/2/15 06:09,North Kingstown,RI,Light,30 seconds,2 separate sightings of medium brightness star...,1/7/15,http://www.nuforc.org/webreports/116/S116422.html,None
35656,1/2/15 00:05,Durham,NC,Fireball,20 seconds,3 red orbs traveling quickly across the night ...,1/7/15,http://www.nuforc.org/webreports/116/S116435.html,None
35677,1/1/15 05:45,Kailua Kona,HI,Disk,2 minutes,UFO spotted by four on New Years Day at Makala...,1/7/15,http://www.nuforc.org/webreports/116/S116451.html,None
35701,1/1/15 00:04,Oakville,MO,Sphere,6 minutes,15-25 orange lights or orbs traveling from Wes...,1/7/15,http://www.nuforc.org/webreports/116/S116388.html,None
35750,12/31/14 11:00,Everett,WA,Unknown,15 minutes,It looked like what was a jet going straight u...,1/7/15,http://www.nuforc.org/webreports/116/S116170.html,None
35758,12/30/14 19:00,Cromwell,MN,,3 minutes,2 flat looking planes with 3 lights on front.,1/7/15,http://www.nuforc.org/webreports/116/S116180.html,None
35771,12/29/14 19:30,York,PA,Circle,15 minutes,"Lighted objects in the night over York, PA.",1/7/15,http://www.nuforc.org/webreports/116/S116181.html,None
35828,12/27/14 05:00,Frederick,MD,Flash,1 second,Object not seen. Only fleeting light similar ...,1/7/15,http://www.nuforc.org/webreports/116/S116274.html,None
35840,12/26/14 18:42,Lancaster,CA,Unknown,,((HOAX)) Every shape size form there here in ...,1/7/15,http://www.nuforc.org/webreports/116/S116284.html,None


In [5]:
# replace the NaN values in Detail_Summary with the string from Detail column
indList = sightings_raw_df[sightings_raw_df.Detail_Summary.isna()].index
for index in indList:
    sightings_raw_df.loc[index].Detail_Summary = sightings_raw_df.loc[index].Summary

In [5]:
sightings_raw_df.Detail_Summary.isna().any()

False

In [6]:
# sanity check
sightings_raw_df.iloc[indList]

,Date_Time,City,State,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary
35635,1/3/15 20:05,Melbourne Beach,FL,Fireball,5 minutes,"Burning object seen falling, then glowing red,...",1/7/15,http://www.nuforc.org/webreports/116/S116518.html,"Burning object seen falling, then glowing red,..."
35652,1/2/15 06:09,North Kingstown,RI,Light,30 seconds,2 separate sightings of medium brightness star...,1/7/15,http://www.nuforc.org/webreports/116/S116422.html,2 separate sightings of medium brightness star...
35656,1/2/15 00:05,Durham,NC,Fireball,20 seconds,3 red orbs traveling quickly across the night ...,1/7/15,http://www.nuforc.org/webreports/116/S116435.html,3 red orbs traveling quickly across the night ...
35677,1/1/15 05:45,Kailua Kona,HI,Disk,2 minutes,UFO spotted by four on New Years Day at Makala...,1/7/15,http://www.nuforc.org/webreports/116/S116451.html,UFO spotted by four on New Years Day at Makala...
35701,1/1/15 00:04,Oakville,MO,Sphere,6 minutes,15-25 orange lights or orbs traveling from Wes...,1/7/15,http://www.nuforc.org/webreports/116/S116388.html,15-25 orange lights or orbs traveling from Wes...
35750,12/31/14 11:00,Everett,WA,Unknown,15 minutes,It looked like what was a jet going straight u...,1/7/15,http://www.nuforc.org/webreports/116/S116170.html,It looked like what was a jet going straight u...
35758,12/30/14 19:00,Cromwell,MN,,3 minutes,2 flat looking planes with 3 lights on front.,1/7/15,http://www.nuforc.org/webreports/116/S116180.html,2 flat looking planes with 3 lights on front.
35771,12/29/14 19:30,York,PA,Circle,15 minutes,"Lighted objects in the night over York, PA.",1/7/15,http://www.nuforc.org/webreports/116/S116181.html,"Lighted objects in the night over York, PA."
35828,12/27/14 05:00,Frederick,MD,Flash,1 second,Object not seen. Only fleeting light similar ...,1/7/15,http://www.nuforc.org/webreports/116/S116274.html,Object not seen. Only fleeting light similar ...
35840,12/26/14 18:42,Lancaster,CA,Unknown,,((HOAX)) Every shape size form there here in ...,1/7/15,http://www.nuforc.org/webreports/116/S116284.html,((HOAX)) Every shape size form there here in ...


In [37]:
# essential functions for cleaning text

def spell_check(text):
    spell = SpellChecker()
    textList = text.split(" ")
    misspelled = spell.unknown(textList)
    for i in range(len(textList)):
        if textList[i] in misspelled:
            textList[i] = spell.correction(textList[i])
    return " ".join(textList)
       
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(w) for w in text.split(' ')])

def expand_contractions(text):
    expanded_words = []    
    for word in text.split():
        expanded_words.append(contractions.fix(word))   

    return ' '.join(expanded_words)

def clean_text(text):
    stop = set(nltk.corpus.stopwords.words('english'))
    cleaned = expand_contractions(text.lower())
    tokens = word_tokenize(cleaned)
    cleaned = ' '.join([w for w in tokens if not w in stop])
    cleaned = lemmatize_text(cleaned)
    return cleaned

In [8]:
# put everything in lowercase
sumDocs = [s.lower() for s in sightings_raw_df.Detail_Summary]

In [9]:
# remove digits
sumDocs = [re.sub(r'\d+', "", s) for s in sumDocs]

In [10]:
# remove newlines
sumDocs = [re.sub(r'\n', "", s) for s in sumDocs]

In [11]:
# remove punctuation and special symbols
sumDocs = [re.sub(r'[\(\)\[\]\^\$\+\*\.\?\/!@#%&{}\'\",;:]', "", s) for s in sumDocs]

In [12]:
# replace hyphen with whitespace
sumDocs = [re.sub(r'-', " ", s) for s in sumDocs]

In [13]:
# remove whitespaces from the beginning and end of strings
sumDocs = [s.strip() for s in sumDocs]

In [14]:
# replace all instances of multiple consecutive whitespaces with a single whitespace
sumDocs = [re.sub(r' +', " ", s) for s in sumDocs]

In [41]:
# replace misspelled words with correctly spelled words
sumDocs = [spell_check(s) for s in sumDocs]

KeyboardInterrupt: 

In [ ]:
# expand contractions and eliminate stop words
sumDocs = [clean_text(s) for s in sumDocs]

In [ ]:
# lemmatize
sumDocs = [lemmatize_text(s) for s in sumDocs]

In [39]:
sumDocs[36015:36020]

['bright greenish light rose up stopped then shot across skysitting at my job site before work when a bright greenish light came up from behind the building line stopped and then shot across the sky im a northeast heading leaving a greenish trail',
 'my wife and i were on our way to work she said whats that i turned to see a triangular very dark in color probably black shaped object it flew hovered extremely slow just over the powerlines it had two constant lights in the bottom center and flashing lights on its extreme edges it was massive it made no noise to be heard our first thought was at that speed and altitude that it was on a controlled crash coarse for a field a half mile away no crash reported we did not know who to tell or what to do',
 'large craft triangle size with rainbow lights flashing around it hovering over the freeway off nw council it was hovering over two police officers one was in a truck the other in a car it just hovered over them with a craft niger then both gr

In [8]:
model = gensim.models.Word2Vec(sumDocs, window=10, min_count=2, workers=10)
model.train(sumDocs,total_examples=len(sumDocs),epochs=10)

(115876365, 157244240)

In [14]:
model.wv.most_similar (positive="light", topn=5)

[('orb', 0.6941230893135071),
 ('dot', 0.6320801973342896),
 ('flash', 0.6174101829528809),
 ('glow', 0.6169379949569702),
 ('object', 0.6047328114509583)]

In [11]:
model.wv.similarity(w1="orange",w2="red")

0.42460892